In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

As a beginner in ML and python, probably this is my 3rd data project. And any feedback would be appretiated. Anyway here's my shot.

First of all, import packages to use later.


In [ ]:
# Packages for reading, visualization and manipulating data
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
from scipy import stats

#scaling and spliting
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#ML algorythms
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier


#performance metrics
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
#read data ,Data examination, clearing NaN values
file_path = '/kaggle/input/pima-indians-diabetes-database/diabetes.csv'
df = pd.read_csv(file_path)

df.describe().T

No missing values in the dataset. However, it is not possible that someone has 0 glucose,bloodpressue, skinthickness , insulin and BMI scores. Therefore it is needed to remove outliners (by changing 0's to NaN and refilling with the column value or by removing Z values below 3; I choose removing Z treshhold method)

In [ ]:
z = np.abs(stats.zscore(df))
threshold = 3
np.where(z>threshold)
df = df[(z < threshold).all(axis=1)]
df.describe().T

Now all columns except Insulin and SkinThickness have no 0 values. Replaces 0's with means of the columns.

In [ ]:
df.loc[(df.SkinThickness<5)& (df.Outcome==0), 'SkinThickness']=int(df[(df.Outcome==0)]['SkinThickness'].mean())
df.loc[(df.SkinThickness<5)& (df.Outcome==1), 'SkinThickness']=int(df[(df.Outcome==1)]['SkinThickness'].mean())
df.loc[(df.Insulin==0)& (df.Outcome==0), 'Insulin']=int(df[(df.Outcome==0)]['Insulin'].mean())
df.loc[(df.Insulin==0)& (df.Outcome==1), 'Insulin']=int(df[(df.Outcome==1)]['Insulin'].mean())
df.describe().T

I think we are good to go.

In [ ]:
#Visiuals
for i in df.columns:
    sns.histplot(x=i, hue= 'Outcome', data=df)
    plt.title(i)
    plt.show()
    
sns.countplot(df['Outcome'])

From the histplots it can be considered that the data is not distributed normally. For the data not disturbed normally, StandardScaler could be used.

In [ ]:
ss = StandardScaler()
train = df.iloc[:,0:8]
train = ss.fit_transform(train)
test = df['Outcome']

In [ ]:
#correlations between variables
corr = df.corr()
sns.heatmap(corr)
plt.show()

In [ ]:
#split file for testing and training
x_train, x_test, y_train, y_test = train_test_split(train,test, train_size=0.7,
                                                    random_state= 42)

In [ ]:
#Machine Learning models and testing
models = {"LR": LogisticRegression(),
          "NB": GaussianNB(),
          "KNN" : KNeighborsClassifier(),
          "DT" : DecisionTreeClassifier(),
          "SVM rbf": SVC(),
          "SVM linear" : SVC(kernel='linear'),
          'LDA' : LinearDiscriminantAnalysis(),
          'RFC' : RandomForestClassifier(),
          'BGC' : BaggingClassifier(),
          'ABC' : AdaBoostClassifier(),
          'GBC' : GradientBoostingClassifier(),
          'DTC' : DecisionTreeClassifier(),
          }
for test, clf in models.items():
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    acc = accuracy_score(y_test,y_pred)
    print( test + ' scores')
    print(acc)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print('*' * 100)

It can be assumed that Random Forest Classifier is a good option to predict Diabetus Mellitus. 

PS: Please rekt me for me to learn. Thank you